In [ ]:
# Install required packages
!pip uninstall -y numpy opencv-python opencv-python-headless mediapipe protobuf > /dev/null 2>&1
!pip install --upgrade pip > /dev/null 2>&1
!pip install numpy==1.26.4 opencv-python-headless==4.8.0.76 mediapipe==0.10.10 protobuf==3.20.3

In [ ]:


# Import libraries
import os
import cv2
import json
import numpy as np
from google.colab import drive
import mediapipe as mp

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Initialize MediaPipe
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands

# Define video paths
video_paths = [
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr1_NIE.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/JoeSupportSide.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/JoeStrongSide.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr2_NIE.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te1_1.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te2_2.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te2_1.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te2_2.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te3_1.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te3_2.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te4_1.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te4_2.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te5_1.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te5_2.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te6_1.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te6_2.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te7_1.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te7_2.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te8_1.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te8_2.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te9_1.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te9_2.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te10_1.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te10_2.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te11_1.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Novice/Videos/Te11_2.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr3_Anticipating.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr4_TooLittleTriggerFinger.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr5_TooMuchTriggerFinger.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr6_OvergrippingWithPrimaryHand.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr7_OvergrippingWithSecondaryHand.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr8_BreakingWristUp.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr9_CheckingTargetOverSightsBetweenShots.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr10_JerkingTrigger.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr11_LimpWristing.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr13_SightsIncorrectlyAlligned.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr12_GrippingTooLow.MOV",
        "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Extra Data/Experienced/Video/Tr14_IncorrectHandPlacement.MOV",
]

# Output directories
graph_output_dir = "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Processed_Graph_Data"
frames_output_dir = "/content/drive/MyDrive/E-RAU(DB)/MA680/data/Shooting/Processed_Frames"

# Create directories
os.makedirs(graph_output_dir, exist_ok=True)
os.makedirs(frames_output_dir, exist_ok=True)

# GRAPH STRUCTURE DEFINITION

def create_graph_structure():
    """Define the anatomical connections between keypoints"""
    pose_edges = [
        (0,1),(1,2),(2,3),(3,7),(0,4),(4,5),
        (5,6),(6,8),(9,10),(11,12),(11,13),
        (13,15),(15,17),(15,19),(15,21),(17,19),
        (12,14),(14,16),(16,18),(16,20),(16,22),
        (18,20),(11,23),(12,24),(23,24),(23,25),
        (24,26),(25,27),(26,28),(27,29),(28,30),
        (29,31),(30,32)
    ]

    hand_edges = [
        (0,1),(1,2),(2,3),(3,4),(0,5),(5,6),
        (6,7),(7,8),(0,9),(9,10),(10,11),(11,12),
        (0,13),(13,14),(14,15),(15,16),(0,17),
        (17,18),(18,19),(19,20)
    ]

    return {
        'pose_edges': pose_edges,
        'hand_edges': hand_edges,
        'offsets': {
            'pose': 0,
            'left_hand': 33,
            'right_hand': 54
        }
    }

# KEYPOINT EXTRACTION

def extract_keypoints(frame, pose, hands):
    """Extract pose and hand keypoints from a frame"""
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pose_results = pose.process(frame_rgb)
    hand_results = hands.process(frame_rgb)

    keypoints = []
    visibility = []

    # Pose (33 points)
    if pose_results.pose_landmarks:
        for lm in pose_results.pose_landmarks.landmark:
            keypoints.append([lm.x, lm.y, lm.z])
            visibility.append(lm.visibility)
    else:
        keypoints.extend([[0,0,0] for _ in range(33)])
        visibility.extend([0] * 33)

    # Hands (21 points each)
    hand_status = {'left': False, 'right': False}

    if hand_results.multi_hand_landmarks:
        for hand_landmarks, handedness in zip(hand_results.multi_hand_landmarks,
                                            hand_results.multi_handedness):
            hand_type = handedness.classification[0].label.lower()
            hand_status[hand_type] = True

            for lm in hand_landmarks.landmark:
                keypoints.append([lm.x, lm.y, lm.z])
                visibility.append(1.0)

    # Fill missing hands
    for hand_type in ['left', 'right']:
        if not hand_status[hand_type]:
            keypoints.extend([[0,0,0] for _ in range(21)])
            visibility.extend([0] * 21)

    return np.array(keypoints), np.array(visibility)

# VIDEO PROCESSING FUNCTION

def process_video(video_path, output_dir, frames_dir=None, interval=10):
    """Process video into graph data while optionally saving original frames"""
    if not os.path.exists(video_path):
        print(f"Video not found: {video_path}")
        return

    video_name = os.path.splitext(os.path.basename(video_path))[0]
    graph_video_dir = os.path.join(output_dir, video_name)
    os.makedirs(graph_video_dir, exist_ok=True)

    # Create frames directory if specified
    if frames_dir:
        frames_video_dir = os.path.join(frames_dir, video_name)
        os.makedirs(frames_video_dir, exist_ok=True)

    graph_config = create_graph_structure()
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = 0
    processed_count = 0

    with mp_pose.Pose(
        static_image_mode=False,
        model_complexity=2,
        enable_segmentation=False,
        min_detection_confidence=0.5
    ) as pose, mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as hands:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            if frame_count % interval == 0:
                # Save original frame if requested
                frame_path = None
                if frames_dir:
                    frame_path = os.path.join(frames_video_dir, f"frame_{frame_count:04d}.jpg")
                    cv2.imwrite(frame_path, frame)

                # Extract keypoints
                keypoints, visibility = extract_keypoints(frame, pose, hands)

                # Build graph structure
                edges = graph_config['pose_edges'].copy()
                for hand_type in ['left_hand', 'right_hand']:
                    offset = graph_config['offsets'][hand_type]
                    edges.extend([(offset+a, offset+b) for (a,b) in graph_config['hand_edges']])

                # Prepare graph data
                graph_data = {
                    'nodes': keypoints.tolist(),
                    'edges': edges,
                    'visibility': visibility.tolist(),
                    'frame': frame_count,
                    'timestamp': frame_count / fps,
                    'video': video_name
                }

                # Add frame reference if saved
                if frame_path:
                    graph_data['original_frame'] = frame_path

                # Save graph data
                graph_path = os.path.join(graph_video_dir, f"frame_{frame_count:04d}.json")
                with open(graph_path, 'w') as f:
                    json.dump(graph_data, f, indent=2)

                processed_count += 1
                print(f"Processed frame {frame_count}", end='\r')

            frame_count += 1

        cap.release()
        print(f"\nFinished processing {video_name}: {processed_count} frames extracted")

# MAIN PROCESSING LOOP


for video_path in video_paths:
    print(f"\nProcessing {os.path.basename(video_path)}...")
    process_video(
        video_path=video_path,
        output_dir=graph_output_dir,
        frames_dir=frames_output_dir,  # Set to None to disable frame saving
        interval=10
    )

print("\nAll videos processed successfully")